In [1]:
import findspark
findspark.init('/home/purvil/spark-2.4.3-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Aggregation').getOrCreate()

* Using it we can run queries against views or table, even we can use system and user function to optimize workloads.
* The Hive metastore is the way in which Hive maintains table information for use across sessions. With Spark SQL, you can connect to your Hive metastore, and access table metadata to reduce file listing.

In [3]:
spark.sql("SELECT 1 + 1").show()

+-------+
|(1 + 1)|
+-------+
|      2|
+-------+



* Above command returns dataframe.

### Catalog
* Abstraction for storage metadat about data stored in table. It lists tables, database and functions.

### Tables
* Equivalent to DataFrame.
* Table alwyas has data. There is no longer a temporary table. Only view which does not contains data.
* Table can be managed and unmanaged.
    - When we define table from files on disk it is unmanaged
    - Define dtable from dataframe using saveAsTable is managed table.

### Describe table
```
DESCRIBE TABLE flight_csv

SHOW PARTITIONS partitioned_flights
```

* We can also cache the table using `CACHE TABLE flights` and `UNCACHE TABLE flights`

### Views
* Specifies set of transformation on top of existing table.
* View can be global, set to database or per session

* To user view is displayed as tables, rather than rewriting all of the data to a new location, they simply perform a transformation on the source data at query time.
* Transformation can be filter, select, group by, rollup.


```
CREATE VIEW just_usa AS
SELECT * FROM flights WHERE dest_country_name = "united states"
```

* Temorary view are only for session

```
CREATE TEMP VIEW name_of_view AS
```

* Global Temp view : Available in entire spark application
```
CREATE GLOBAL TEMP VIEW name AS
```

* Overwrite existing view
```
CREATE OR REPLACE TEMP VIEW just_usa_name AS
```

* We can query view just like table. View is like creating new DF from existing.

* To view execution plan
```
EXPLAIN SELECT * FROM use_view
```

* List database
```
SHOW DATABASES
```

### Complex Type
* struct, list, map

#### Struct
* Way to create or query nested data in Spark. To create simply wrap nset of columns in `()`

```
CREATE VIEW IF NOT EXISTS nested_data AS
SELECT (DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME) as country, count FROM flights
```

* To query 
```
SELECT country.DEST_COUNTRY_NAME, count FROM nested_data
```

#### Lists
* `collect_list` will create list of values. `collect_set` will create array without duplicate.
* Both can work with aggregation.
```
SELECT DEST_COUNTRY_NAME as new_name, collect_list(count) as flight_count, collect_set(ORIGIN_COUTNRY_NAME) as origin_set
FROM flights GROUP BY DEST_COUNTRY_NAME
```

* To create array manually

```
SELCT DEST_COUNTRY_NAME, ARRAY(1,2,3) FROM flights
```

```
SELECT DEST_COUNTRY_NAME as new_name, collect_list(count)[0] 
FROM flights GROUP BY DEST_COUNTRY_NAME
```

* To convert array back to row use, `explode`

```
CREATE OR REPLACE TEMP VIEW flight_agg AS
SELECT DEST_COUNTRY_NAME, collect_list(count) as collected_counts FROM flights GROUP BY DEST_COUNTRY_NAME
```

```
SELECT explode(collected_counts), DEST_COUNTRY_NAME FROM flights_agg
```

### Function
* To find available list of functions
```
SHOW FUNCTIONS
```

* To only view system functions
```
SHOW SYSTEM FUNCTIONS
```

```
SHOW USER FUNCTIONS
```
* Will show user defined functions

```
SHOW FUNCTIONS "s*"

SHOW FUNCTIONS LIKE "collect*"
```

### Uncorrelated predicate subqueries
* No info from outer scope

```
SELECT dest_country FROM FLIGHT
GROUPBY dest_country ORDER BY sum(count) DESC LIMIT 5
```

```
SELECT * FROM FLIGHTS
WHERE origin_country_name 
IN (SELECT dest_country FROM FLIGHT
GROUPBY dest_country ORDER BY sum(count) DESC LIMIT 5)
```

```
SELECT *, (SELECT max(count) FROM flights) AS max FROM flights
```

### Correlated predicate subqueries
* Uses information of outer scope in inner subqueries

```
SELECT * FROM flights f1
WHERE EXISTS (SELECT 1 FROM flights f2 WHERE f1.dest_country_name = f2.origin_country_name)
AND EXISTS (SELECT 1 FROM flights f2 WHERE f2.dest_country_name = f1.origin_country_name)
```